In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emojis5/train_emoji.csv
/kaggle/input/emojis5/test_emoji.csv
/kaggle/input/glove6b50dtxt/glove.6B.50d.txt


In [2]:
import emoji
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical

from keras.layers import *
from keras.models import Sequential

print('Imports ready !')

Imports ready !


In [3]:
train = pd.read_csv('../input/emojis5/train_emoji.csv', header = None)
test = pd.read_csv('../input/emojis5/test_emoji.csv', header = None)

print(train.shape, " ", test.shape)

(132, 4)   (56, 2)


In [4]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [5]:
emoji_dictionary

{'0': '❤️',
 '1': ':baseball:',
 '2': ':beaming_face_with_smiling_eyes:',
 '3': ':downcast_face_with_sweat:',
 '4': ':fork_and_knife:'}

In [6]:
x_train = train.values[:, 0]
y_train = to_categorical(train.values[:, 1])

x_test = test.values[:, 0]
y_test = to_categorical(test.values[:, 1])

In [7]:
y_train[0:5]

array([[0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [8]:
embeddings = {}
with open('../input/glove6b50dtxt/glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split() # splits the word and coeff
        word = values[0] # word
        coeffs = np.asarray(values[1:],dtype='float32') # makes a word vector of len 50 for each word
        embeddings[word] = coeffs

In [9]:
x_train[0:5]

array(['never talk to me again', 'I am proud of your achievements',
       'It is the worst day in my life', 'Miss you so much',
       'food is life'], dtype=object)

In [10]:
def getOutputEmbeddings(X):
    
    embedding_matrix_output = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split() # get a list of words of the sentence
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings[X[ix][jx].lower()]
            
    return embedding_matrix_output

In [11]:
emb_x_train = getOutputEmbeddings(x_train)
emb_x_test = getOutputEmbeddings(x_test)

In [12]:
emb_x_train.shape

(132, 10, 50)

In [13]:
emb_x_test.shape

(56, 10, 50)

In [14]:
model = Sequential()
model.add(LSTM(64,input_shape = (10,50),return_sequences=True)) # to create a stacked LSTM model
model.add(Dropout(0.4))
model.add(LSTM(64,input_shape = (10,50)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

2022-11-18 03:30:24.004304: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 64)            29440     
_________________________________________________________________
dropout (Dropout)            (None, 10, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
_________________________________________________________________
activation (Activation)      (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
____________________________________________________

In [15]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
model.fit(emb_x_train,y_train,batch_size=32,epochs=40,shuffle=True,validation_split=0.1)

Epoch 1/40


2022-11-18 03:30:24.971824: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


4/4 [==============================] - 6s 385ms/step - loss: 1.5775 - acc: 0.2712 - val_loss: 1.5971 - val_acc: 0.2857
Epoch 2/40
4/4 [==============================] - 0s 38ms/step - loss: 1.5340 - acc: 0.3475 - val_loss: 1.6098 - val_acc: 0.2857
Epoch 3/40
4/4 [==============================] - 0s 38ms/step - loss: 1.5126 - acc: 0.3475 - val_loss: 1.6396 - val_acc: 0.2857
Epoch 4/40
4/4 [==============================] - 0s 35ms/step - loss: 1.4732 - acc: 0.3983 - val_loss: 1.6384 - val_acc: 0.1429
Epoch 5/40
4/4 [==============================] - 0s 33ms/step - loss: 1.4073 - acc: 0.4576 - val_loss: 1.5817 - val_acc: 0.2857
Epoch 6/40
4/4 [==============================] - 0s 35ms/step - loss: 1.3904 - acc: 0.4492 - val_loss: 1.5210 - val_acc: 0.2857
Epoch 7/40
4/4 [==============================] - 0s 34ms/step - loss: 1.2951 - acc: 0.5254 - val_loss: 1.4447 - val_acc: 0.2857
Epoch 8/40
4/4 [==============================] - 0s 34ms/step - loss: 1.2286 - acc: 0.5254 - val_loss: 1.3

In [16]:
model.evaluate(emb_x_test,y_test)

2/2 [==============================] - 0s 10ms/step - loss: 1.6903 - acc: 0.6071


[1.6902681589126587, 0.6071428656578064]

In [17]:
pred = model.predict(emb_x_test)
pred=np.argmax(pred,axis=1)

In [18]:
print("Sentence      : Actual       Prediction")
for i in range(10):
    print(' '.join(x_test[i]), end = " : ")
    print(emoji.emojize(emoji_dictionary[str(np.argmax(y_test[i]))]), end = " ")
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

Sentence      : Actual       Prediction
I want to eat : 🍴 🍴
he did not answer : 😓 😓
he got a raise : 😁 😁
she got me a present : ❤️ 😁
ha ha ha it was so funny : 😁 😁
he is a good friend : ❤️ 😁
I am upset : ❤️ 😓
We had such a lovely dinner tonight : ❤️ 😁
where is the food : 🍴 🍴
Stop making this joke ha ha ha : 😁 😁
